# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [172]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [173]:
# identify the columns from each table you want using the table.column syntax.
# using the FROM and INNER JOIN to identify which tables we need to combine - can also set aliases for the tables with AS.
# using the ON statement with an equality (=) to identify on which column we join these tables, then LIMIT the ouput.
query1 = "SELECT tracks.Name, albums.Title FROM tracks INNER JOIN albums ON tracks.Albumid = albums.Albumid LIMIT 5;"
table1 = %sql $query1
display(table1)

 * sqlite:///chinook.sqlite
Done.


Name,Title
For Those About To Rock (We Salute You),For Those About To Rock We Salute You
Put The Finger On You,For Those About To Rock We Salute You
Let's Get It Up,For Those About To Rock We Salute You
Inject The Venom,For Those About To Rock We Salute You
Snowballed,For Those About To Rock We Salute You


In [174]:
# identify the column names and select, using the count aggregating function count the customerids and assign the an alias.
# using FROM grab the customer table, group it by the country and order it by the customer count in descending order.
query2 = "SELECT COUNT(Customerid) AS cust_count, Country FROM customers GROUP BY Country ORDER BY cust_count DESC;"
table2 = %sql $query2
display(table2)

 * sqlite:///chinook.sqlite
Done.


cust_count,Country
13,USA
8,Canada
5,France
5,Brazil
4,Germany
3,United Kingdom
2,Portugal
2,India
2,Czech Republic
1,Sweden


In [175]:
#3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
#   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
query3 = "SELECT COUNT(tracks.Name) AS track_count, genres.Name FROM tracks INNER JOIN genres ON genres.Genreid = tracks.Genreid GROUP BY genres.Name ORDER BY track_count DESC LIMIT 5;"
table3 = %sql $query3
display(table3)




 * sqlite:///chinook.sqlite
Done.


track_count,Name
1297,Rock
579,Latin
374,Metal
332,Alternative & Punk
130,Jazz


In [176]:
# first create the main query on the customers column:
# using the key = CustomerId, and combining the First and Last names into a single column FullName
# the final column is the result of the subquery which averages the Invoice totals for each customer.
# then we create the subquery from the invoices table to JOIN with the main query:
# using the invoices total we average the invoice totals and group averages by their customerId assigning this subquery to an alias.
# after joining on customerId we organize average invoice total in descending order.
query4 = "SELECT customers.CustomerId, customers.FirstName || ' ' || customers.LastName AS FullName, subquery.Average_Invoice_Total FROM customers INNER JOIN (SELECT ROUND(AVG(invoices.Total), 2) AS Average_Invoice_Total, invoices.CustomerId, invoices.InvoiceId FROM invoices GROUP BY CustomerId) AS subquery ON customers.CustomerId = subquery.CustomerId ORDER BY Average_Invoice_Total DESC LIMIT 10;"
table4 = %sql $query4
display(table4)

 * sqlite:///chinook.sqlite
Done.


CustomerId,FullName,Average_Invoice_Total
6,Helena Holý,7.09
26,Richard Cunningham,6.8
57,Luis Rojas,6.66
45,Ladislav Kovács,6.52
46,Hugh O'Reilly,6.52
24,Frank Ralston,6.23
28,Julia Barnett,6.23
37,Fynn Zimmermann,6.23
59,Puja Srivastava,6.11
7,Astrid Gruber,6.09


In [177]:
# similar to query 4...
# first create the main query on the customers column:
# using the key = CustomerId, and combining the First and Last names into a single column FullName
# the final column is the result of the subquery which sum of the Invoice totals for each customer.
# then we create the subquery from the invoices table to JOIN with the main query:
# using the invoices total sum the invoices and group sums by their customerId assigning this subquery to an alias.
# after joining on customerId we organize average invoice total in descending order and limit to top 10 results.
query5 = "SELECT customers.CustomerId, customers.FirstName || ' ' || customers.LastName AS FullName, subquery.Purchase_Total FROM customers INNER JOIN (SELECT invoices.CustomerId, ROUND(SUM(invoices.Total), 2) AS Purchase_Total FROM invoices GROUP BY CustomerId) AS subquery ON customers.CustomerId = subquery.CustomerId ORDER BY Purchase_Total DESC LIMIT 10;"
table5 = %sql $query5
display(table5)


 * sqlite:///chinook.sqlite
Done.


CustomerId,FullName,Purchase_Total
6,Helena Holý,49.62
26,Richard Cunningham,47.62
57,Luis Rojas,46.62
45,Ladislav Kovács,45.62
46,Hugh O'Reilly,45.62
24,Frank Ralston,43.62
28,Julia Barnett,43.62
37,Fynn Zimmermann,43.62
7,Astrid Gruber,42.62
25,Victor Stevens,42.62
